### Расчеты параметров модели для эмуляции 


In [2]:
import os, time, math, pathlib, bs4
from bs4 import BeautifulSoup
from sympy import sympify, Symbol, symbols, Matrix, init_printing, pretty
init_printing()

In [3]:
BOX = 'box'
SPHERE = 'sphere';
CYLINDER = 'cylinder'

class Geometry:
    def __init__(self, trait):
        self.__trait = trait
    def checkNodeType(self, node):
        if not isinstance(node, bs4.element.Tag):
            raise Exception('Node type not applicable')
    def getTrait(self):
        return self.__trait
    def isTrait(self, trait: str):
        return self.__trait == trait

class GeometryBox(Geometry):
    def __init__(self, node):
        super().__init__(BOX)
        super().checkNodeType(node)
        sizes = node['size'].split(" ")
        if len(sizes) != 3:
            Exception("Invalid size array for geometry box")
        self.__width = float(sizes[0])
        self.__depth = float(sizes[1])
        self.__height = float(sizes[2])     
    def getWidth(self):
        return float(self.__width)
    def getHeight(self):
        return float(self.__height)
    def getDepth(self):
        return float(self.__depth)
    def __str__(self):
        return "w:{:.2f}/d:{:.2f}/h:{:.2f}".format(self.getWidth(), self.getDepth(), self.getHeight())

class GeometrySphere(Geometry):
    def __init__(self, node):
        super().__init__(SPHERE)
        super().checkNodeType(node)
        self.__radius = float(node['radius'])
    def getRadius(self):
        return self.__radius
    def __str__(self):
        return "r:{:.2f}".format(self.getRadius())

class GeometryCylinder(Geometry):
    def __init__(self, node):
        super().__init__(CYLINDER)
        super().checkNodeType(node)
        self.__length = float(node['length'])
        self.__radius = float(node['radius'])
    def getLength(self):
        return float(self.__length)
    def getRadius(self):
        return float(self.__radius)
    def __str__(self):
        return "l:{:.2f}/r:{:.2f}".format(self.getLength(), self.getRadius())

class Inertial:
    __mass = xx = xy = xz = yy = yz = zz = .0
    def __init__(self, inertial):
        if not isinstance(inertial, bs4.element.Tag):
            raise Exception('Type of inertial node not applicable')
        if inertial.mass:
            self.__mass = float(inertial.mass['value'])
        if not inertial.inertia is None:
            self.xx = float(inertial.inertia['ixx'])
            self.xy = float(inertial.inertia['ixy'])
            self.xz = float(inertial.inertia['ixz'])
            self.yy = float(inertial.inertia['iyy'])
            self.yz = float(inertial.inertia['iyz'])
            self.zz = float(inertial.inertia['izz'])
    def getMass(self):
        return float(self.__mass)
    def getData(self):
        return [float(self.xx), float(self.xy), float(self.xz),
                float(self.yy), float(self.yz), float(self.zz)]
    def __str__(self):
        return "m:{:.4f}/inertials:[{}]".format(self.getMass(), str(self.getData())[1:-1])

In [13]:
dirPath = os.path.dirname(os.getcwd())
xmlRelConfigFilePath = "robot_frame/description/robot_description.xml"
xmlConfigFullFilePath = os.path.join(dirPath, xmlRelConfigFilePath)

if not pathlib.Path(xmlConfigFullFilePath).is_file():
    raise Exception(f"Unable to find config file ['{xmlConfigFullFilePath}']")

with open(xmlConfigFullFilePath, 'r') as fl:
    data = fl.read()

rootData = BeautifulSoup(data, "xml")

geometries = dict()
inertials = dict()

for link in rootData.find_all('link'):
    visual = link.visual
    inertial = link.inertial
    linkName = link['name']
    if not visual is None:
        if not visual.geometry.sphere is None:
            sphere = visual.geometry.sphere
            geometries[linkName] = GeometrySphere(sphere)
        elif not visual.geometry.cylinder is None:
            cylinder = visual.geometry.cylinder
            geometries[linkName] = GeometryCylinder(cylinder)
        elif not visual.geometry.box is None:
            box = visual.geometry.box
            geometries[linkName] = GeometryBox(box)
    if not inertial is None:
        inertials[linkName] = Inertial(inertial)
#
# Расчет данных для конфигурирования контроллера дифференциального привода
#
wheelDiameter = geometries['front_left_wheel'].getRadius()*2
wheelScrewLen = geometries['front_left_wheel'].getLength()/8
wheelSeparation = geometries['front_axle'].getLength() - geometries['front_left_wheel'].getLength()/2 - wheelScrewLen 
#
# Расчет общей массы изделия
#
totalMass = float(0)
for link in rootData.find_all('link'):
    inertial = link.inertial
    if not inertial is None:
        mass = inertial.mass
        if not mass is None:
            totalMass += float(mass['value'])
#
# Расчет фактической длины, ширины и высоты
#
totalHeight = \
    geometries['front_left_wheel'].getRadius() + \
    geometries['front_axle'].getRadius()*2 + \
    geometries['chassis'].getHeight() + \
    geometries['box_panel_back'].getHeight() + \
    geometries['hinge_cover'].getRadius()

totalWidth = \
    geometries['back_left_wheel'].getLength() + \
    geometries['back_right_wheel'].getLength() + \
    geometries['box_panel_back'].getWidth() + \
    geometries['box_gripper_holder_left'].getLength() + \
    geometries['manipulator_1st_left'].getLength() + \
    geometries['manipulator_1st_left'].getRadius()

totalLength = \
    geometries['chassis'].getWidth() + \
    geometries['box_ladle_front'].getDepth()/2 + \
    geometries['box_bracket_holder_left'].getDepth() + \
    (geometries['back_left_wheel'].getRadius() - round(geometries['chassis'].getWidth()/8, 3))*2

totalLengthWithManip = \
    geometries['box_panel_left'].getDepth()/2 + \
    geometries['box_panel_back'].getDepth() + \
    geometries['back_left_wheel'].getRadius() - round(geometries['chassis'].getWidth()/8, 3) + \
    geometries['articulation_1st_left'].getRadius() + \
    geometries['manipulator_2nd_left'].getLength() + \
    geometries['manipulator_3rd_left'].getLength() + \
    geometries['articulation_end_effector_left'].getRadius() + \
    0.06 # gripper lenght = 0.06

print("Диаметр колес: {:.3f} (в метрах)".format(wheelDiameter))
print("Расстояние между левым и правым колесами: {:.3f} (в метрах)".format(wheelSeparation))  
print("\nОбщий вес изделия: {:.1f} кг.".format(totalMass))
print("Общая высота изделия: {:.2f} метра.".format(totalHeight))
print("Общая ширина изделия: {:.2f} метра.".format(totalWidth))
print("Общая длина изделия: {:.2f} метра. Длина с манипуляторами: {:.2f} метра.".format(totalLength, totalLengthWithManip))

Диаметр колес: 0.185 (в метрах)
Расстояние между левым и правым колесами: 0.485 (в метрах)

Общий вес изделия: 25.5 кг.
Общая высота изделия: 0.48 метра.
Общая ширина изделия: 0.54 метра.
Общая длина изделия: 0.67 метра. Длина с манипуляторами: 0.72 метра.


#### Проверка параметров модули изделия

In [5]:
# символы/параметры описывающие модель
mass = symbols('m', real=True)
height = symbols('h', real=True)
width = symbols('w', real=True)
depth = symbols('d', real=True)
radius = symbols('r', real=True)
length = symbols('l', real=True)

In [6]:
# Тензор инерции для объека кубической формы
ixx = sympify("(1/12)*m*(d**2 + h**2)")
iyy = sympify("(1/12)*m*(w**2 + h**2)")
izz = sympify("(1/12)*m*(w**2 + d**2)")
ixy = ixz = iyz = iyx = izy = izx = 0.0

CT = Matrix([
    [ixx, ixy, ixz], 
    [iyx, iyy, iyz], 
    [izx, izy, izz], 
])

# Тензор инерции для объекта цилиндр
ixx = sympify("(1/12)*m*(3*(r**2) + l**2)")
iyy = sympify("(1/12)*m*(3*(r**2) + l**2)")
izz = sympify("(1/2)*m*(r**2)")
ixy = ixz = iyz = iyx = izy = izx = 0.0

TT = Matrix([
    [ixx, ixy, ixz], 
    [iyx, iyy, iyz], 
    [izx, izy, izz], 
])

# Тензор инерции для объекта сферической формы
ixx = iyy = izz = sympify("(2/5)*m*(r**2)")
ixy = ixz = iyz = iyx = izy = izx = 0.0

ST = Matrix([
    [ixx, ixy, ixz], 
    [iyx, iyy, iyz], 
    [izx, izy, izz], 
])

In [7]:
print("Формула рассчета тензора для кубической формы")
CT

Формула рассчета тензора для кубической формы


⎡  ⎛ 2    2⎞                          ⎤
⎢m⋅⎝d  + h ⎠                          ⎥
⎢───────────       0            0     ⎥
⎢     12                              ⎥
⎢                                     ⎥
⎢               ⎛ 2    2⎞             ⎥
⎢             m⋅⎝h  + w ⎠             ⎥
⎢     0       ───────────       0     ⎥
⎢                  12                 ⎥
⎢                                     ⎥
⎢                            ⎛ 2    2⎞⎥
⎢                          m⋅⎝d  + w ⎠⎥
⎢     0            0       ───────────⎥
⎣                               12    ⎦

In [8]:
print("Формула рассчета тензора для цилиндрической формы")
TT

Формула рассчета тензора для цилиндрической формы


⎡  ⎛ 2      2⎞                     ⎤
⎢m⋅⎝l  + 3⋅r ⎠                     ⎥
⎢─────────────        0         0  ⎥
⎢      12                          ⎥
⎢                                  ⎥
⎢                 ⎛ 2      2⎞      ⎥
⎢               m⋅⎝l  + 3⋅r ⎠      ⎥
⎢      0        ─────────────   0  ⎥
⎢                     12           ⎥
⎢                                  ⎥
⎢                                 2⎥
⎢                              m⋅r ⎥
⎢      0              0        ────⎥
⎣                               2  ⎦

In [9]:
print("Формула рассчета тензора для сферической формы")
ST

Формула рассчета тензора для сферической формы


⎡     2                ⎤
⎢2⋅m⋅r                 ⎥
⎢──────    0       0   ⎥
⎢  5                   ⎥
⎢                      ⎥
⎢             2        ⎥
⎢        2⋅m⋅r         ⎥
⎢  0     ──────    0   ⎥
⎢          5           ⎥
⎢                      ⎥
⎢                     2⎥
⎢                2⋅m⋅r ⎥
⎢  0       0     ──────⎥
⎣                  5   ⎦

##### Проверка расчет инерциальных параметров по физическим данным модели 

In [10]:
# Концевой эффектор манипулятора - левый/правый (articulation_end_effector_left/articulation_end_effector_right)
linksName = ['articulation_end_effector_left', 'articulation_end_effector_right']
for linkName in linksName:
    inertia = inertials[linkName]
    values = { 'm': inertia.getMass(), 'r': geometries[linkName].getRadius() }
    ixx.subs(values)
    iyy.subs(values)
    izz.subs(values)
    STR = ST.subs(values)
    
    print("Данные инерции из модели для узла '{}': ixx={:.6f}, iyy={:.6f}, izz={:.6f}.".format(
            linkName,
            inertia.xx,
            inertia.yy,
            inertia.zz,
    ))
    
    print("Рассчитанные данные: ixx={:.6f}, iyy={:.6f}, izz={:.6f}, det={:.12f}.".format(
        STR[0,0],
        STR[1,1],
        STR[2,2],
        STR.det()))
STR

Данные инерции из модели для узла 'articulation_end_effector_left': ixx=0.000013, iyy=0.000013, izz=0.000013.
Рассчитанные данные: ixx=0.000013, iyy=0.000013, izz=0.000013, det=0.000000000000.
Данные инерции из модели для узла 'articulation_end_effector_right': ixx=0.000013, iyy=0.000013, izz=0.000013.
Рассчитанные данные: ixx=0.000013, iyy=0.000013, izz=0.000013, det=0.000000000000.


⎡1.30666666666667e-5           0                    0         ⎤
⎢                                                             ⎥
⎢         0           1.30666666666667e-5           0         ⎥
⎢                                                             ⎥
⎣         0                    0           1.30666666666667e-5⎦

In [11]:
# Крышка люка (box_cover)
linkName = 'box_cover'
box_cover = geometries[linkName]
height = box_cover.getHeight() # meters
width = box_cover.getWidth()  # meters
depth = box_cover.getDepth()  # meters
mass = inertials[linkName].getMass()  # kg

values = { 'm': mass, 'h': height, 'w': width, 'd': depth }
ixx.subs(values)
iyy.subs(values)
izz.subs(values)
CTR = CT.subs(values)

inertia = inertials[linkName]
print("Данные инерции из модели для узла '{}': ixx={:.6f}, iyy={:.6f}, izz={:.6f}.".format(
        linkName,
        inertia.xx,
        inertia.yy,
        inertia.zz,
))

print("Рассчитанные данные: ixx={:.6f}, iyy={:.6f}, izz={:.6f}, det={:.12f}.".format(
    CTR[0,0],
    CTR[1,1],
    CTR[2,2],
    CTR.det()))
CTR

Данные инерции из модели для узла 'box_cover': ixx=0.007449, iyy=0.003464, izz=0.010830.
Рассчитанные данные: ixx=0.003464, iyy=0.007449, izz=0.010830, det=0.000000279460.


⎡0.00346412967021075           0                   0         ⎤
⎢                                                            ⎥
⎢         0           0.00744907407407407          0         ⎥
⎢                                                            ⎥
⎣         0                    0           0.0108298704109515⎦

In [12]:
# Шарнир крышки люка (hinge_cover)
linkName = 'hinge_cover'
mass = inertials[linkName].getMass() # kg
radius = geometries[linkName].getRadius() # meters
length = geometries[linkName].getLength() # meters
values = { 'm': mass, 'r': radius, 'l': length }

ixx.subs(values)
iyy.subs(values)
izz.subs(values)
CTT = TT.subs(values)

inertia = inertials[linkName]
print("Данные инерции из модели для узла '{}': ixx={:.6f}, iyy={:.6f}, izz={:.6f}.".format(
        linkName,
        inertia.xx,
        inertia.yy,
        inertia.zz,
))

print("Рассчитанные данные: ixx={:.6f}, iyy={:.6f}, izz={:.6f}, det={:.12f}.".format(
    CTT[0,0],
    CTT[1,1],
    CTT[2,2],
    CTT.det()))
CTT

Данные инерции из модели для узла 'hinge_cover': ixx=0.002687, iyy=0.002687, izz=0.000040.
Рассчитанные данные: ixx=0.002687, iyy=0.002687, izz=0.000040, det=0.000000000289.


⎡0.00268666666666667           0             0   ⎤
⎢                                                ⎥
⎢         0           0.00268666666666667    0   ⎥
⎢                                                ⎥
⎣         0                    0           4.0e-5⎦